# Etapa 1 - Ver tablas 
**Inspeccion rapida del .sqlite**

In [ ]:
import sqlite3

# Ruta de la base de datos SQLite
db_path = r"C:\Users\Laptop\Desktop\Proyecto compu 2\2019\2019-12.sqlite"

# Conexión a la base de datos SQLite
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Litar las tablas existentes 
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

print("Tablas en la base de datos:")
for t in tables:
    print("-", t[0])

# Cerrar la conexión
conn.close()


# Etapa 2 - Ver columnas y tipos
**Revisamos la estructura de la tabla**


In [9]:
import sqlite3

# Ruta de la base de datos SQLite
db_path = r"C:\Users\Laptop\Desktop\Proyecto compu 2\2019\2019-12.sqlite"

# Conexión a la base de datos SQLite
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Ver columnas de la tabla tripdata
cursor.execute("PRAGMA table_info(tripdata);")
cols = cursor.fetchall()

print("Columnas en tripdata:")
for col in cols:
    # col = (cid, name, type, notnull, dflt_value, pk)
    print(f"- {col[1]} ({col[2]})")

conn.close()

Columnas en tripdata:
- vendorid (FLOAT)
- tpep_pickup_datetime (DATETIME)
- tpep_dropoff_datetime (DATETIME)
- passenger_count (FLOAT)
- trip_distance (FLOAT)
- ratecodeid (FLOAT)
- store_and_fwd_flag (TEXT)
- pulocationid (FLOAT)
- dolocationid (FLOAT)
- payment_type (FLOAT)
- fare_amount (FLOAT)
- extra (FLOAT)
- mta_tax (FLOAT)
- tip_amount (FLOAT)
- tolls_amount (FLOAT)
- improvement_surcharge (FLOAT)
- total_amount (FLOAT)
- congestion_surcharge (FLOAT)


# Etapa 3 - Revisar Zonas 
**Para saber como estan codificadas las zonas**


In [32]:
import pandas as pd

# Ruta del archivo de zonas 
zones_path = r"C:\Users\Laptop\Desktop\nyc-taxi-demanda-2019\nyc-taxi-demanda-2019\data\taxi_zone_lookup.csv"

# Leer el archivo CSV
zones = pd.read_csv(zones_path)

# 2️⃣ Revisar estructura
print("\n📌 Primeras filas de taxi_zone_lookup.csv:")
print(zones.head())

print("\n📋 Columnas disponibles:")
print(zones.columns.tolist())

# 3️⃣ Revisar tipos de datos
print("\n🧾 Tipos de datos:")
print(zones.dtypes)

# 4️⃣ Comprobar si hay nulos o duplicados
print("\n❓ Nulos por columna:")
print(zones.isna().sum())

print("\n🔁 Duplicados por LocationID:")
print(zones['LocationID'].duplicated().sum())



📌 Primeras filas de taxi_zone_lookup.csv:
   LocationID        Borough                     Zone service_zone
0           1            EWR           Newark Airport          EWR
1           2         Queens              Jamaica Bay    Boro Zone
2           3          Bronx  Allerton/Pelham Gardens    Boro Zone
3           4      Manhattan            Alphabet City  Yellow Zone
4           5  Staten Island            Arden Heights    Boro Zone

📋 Columnas disponibles:
['LocationID', 'Borough', 'Zone', 'service_zone']

🧾 Tipos de datos:
LocationID       int64
Borough         object
Zone            object
service_zone    object
dtype: object

❓ Nulos por columna:
LocationID      0
Borough         1
Zone            1
service_zone    2
dtype: int64

🔁 Duplicados por LocationID:
0


In [33]:
# ✅ Etapa 3: Preparar tabla de zonas (lookup)
zones_path = r"C:\Users\Laptop\Desktop\nyc-taxi-demanda-2019\nyc-taxi-demanda-2019\data\taxi_zone_lookup.csv"
zones = pd.read_csv(zones_path)

# Nos quedamos con las columnas necesarias
zones = zones[['LocationID', 'Borough', 'Zone']]

# Verificamos el resultado final
print("\n✅ Tabla de zonas (lookup) lista:")
print(zones.head(10).to_string(index=False))


✅ Tabla de zonas (lookup) lista:
 LocationID       Borough                    Zone
          1           EWR          Newark Airport
          2        Queens             Jamaica Bay
          3         Bronx Allerton/Pelham Gardens
          4     Manhattan           Alphabet City
          5 Staten Island           Arden Heights
          6 Staten Island Arrochar/Fort Wadsworth
          7        Queens                 Astoria
          8        Queens            Astoria Park
          9        Queens              Auburndale
         10        Queens            Baisley Park


# Etapa 4 - Exploracion basica y conteos
**Top zonas pickup/dropoff con nombres**


In [ ]:
import sqlite3
import pandas as pd

# Ruta de la base de datos SQLite
db_path = r"C:\Users\Laptop\Desktop\Proyecto compu 2\2019\2019-12.sqlite"

# Conexión a la base de datos SQLite
conn = sqlite3.connect(db_path)

# Leemos una muestra de datos de la tabla tripdata
query = "SELECT * FROM tripdata LIMIT 100000;"
df = pd.read_sql_query(query, conn)

# Cerramos la conexión
conn.close()

# Verificamos la estructura del DataFrame
print("\n📌 Columnas disponibles en tripdata:")
print(df.columns.tolist())

print("\n Primeras filas de tripdata:")
print(df.head())


In [35]:
# 🧼 Convertimos pulocationid y dolocationid a enteros
df['pulocationid'] = df['pulocationid'].astype(int)
df['dolocationid'] = df['dolocationid'].astype(int)

# 👀 Verificamos rápidamente
print(df[['pulocationid', 'dolocationid']].head())
print(df[['pulocationid', 'dolocationid']].dtypes)

   pulocationid  dolocationid
0           142           116
1           145           145
2           145           145
3           138            25
4           161           237
pulocationid    int64
dolocationid    int64
dtype: object


In [39]:
# Conteos Top de zonas de pickup y dropoff

# Top zonas de recogida (pickup)
pickup_counts = (
    df['pulocationid']
    .value_counts()
    .reset_index()
    .rename(columns={'index': 'LocationID', 'pulocationid': 'LocationID'})
)

# Top zonas de entrega (dropoff)
dropoff_counts = (
    df['dolocationid']
    .value_counts()
    .reset_index()
    .rename(columns={'index': 'LocationID', 'dolocationid': 'LocationID'})
)

print("🚖 Top 15 zonas de recogida (pickup) [IDs]:")
print(pickup_counts.head(15).to_string(index=False))

print("\n📍 Top 15 zonas de entrega (dropoff) [IDs]:")
print(dropoff_counts.head(15).to_string(index=False))

🚖 Top 15 zonas de recogida (pickup) [IDs]:
 LocationID  count
        186   4798
        132   4450
        230   4028
         79   3594
         48   3523
        237   3185
        162   3156
        161   3152
        236   3068
        239   2803
         68   2732
        142   2700
        163   2687
        138   2625
        170   2584

📍 Top 15 zonas de entrega (dropoff) [IDs]:
 LocationID  count
        230   3687
        236   3437
        186   3303
         48   3215
        161   3170
        237   3133
        246   3020
         79   2677
        142   2673
        239   2642
        170   2561
         68   2529
        141   2397
        163   2337
        234   2109
